In [6]:
import pandas as pd
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
from IPython import display
import matplotlib.pyplot as plt

In [7]:
ic = wordnet_ic.ic("ic-brown-resnik-add1.dat")

In [8]:
df = pd.read_csv("data/464_complete_data.csv")
df.drop(columns="Unnamed: 0", inplace=True)
df = df.rename(columns={
    'human_similarity': 'human',
    'gpt_similarity_left': 'gpt_left',
    'gpt_similarity_right': 'gpt_right',
    'gpt_similarity': 'gpt2'
})

In [9]:
df.head()

,word1,word2,human,gpt_left,gpt_right,gpt2,gpt4
0,scholar,academician,5.42,0.850105,0.853856,0.851981,6.0
1,review,critique,5.41,0.822096,0.828070,0.825083,5.0
2,refund,reimbursement,5.41,0.812315,0.810235,0.811275,6.2
3,haven,refuge,5.41,0.669938,0.703791,0.686865,6.9
4,rebel,revolutionary,5.40,0.808106,0.810149,0.809128,5.3


### Most Common (first) Synset Similarity using WUP

In [10]:
def wup_common(word1, word2):
    try:
        synset1 = wn.synsets(word1, pos=wn.NOUN)[0]  # First synset for 'dog'
        synset2 = wn.synsets(word2, pos=wn.NOUN)[0]  # First synset for 'wolf'

        # Compare similarity using Wu-Palmer similarity measure
        return synset1.wup_similarity(synset2)
    except:
        return -1

### Average Similarity using WUP

In [11]:
def wup_average(word1, word2):
    synsets_word1 = wn.synsets(word1, pos=wn.NOUN)
    synsets_word2 = wn.synsets(word2, pos=wn.NOUN)
    scores = []
    for s1 in synsets_word1:
        for s2 in synsets_word2:
            score = s1.wup_similarity(s2)
            if score is not None:
                scores.append(score)
    return sum(scores) / len(scores) if scores else -1

### Most Common (first) Synset Similarity using LIN

In [12]:
def lin_common(word1, word2):
    try:
        synset1 = wn.synsets(word1, pos=wn.NOUN)[0]  # First synset for 'dog'
        synset2 = wn.synsets(word2, pos=wn.NOUN)[0]  # First synset for 'wolf'

        # Compare similarity using Wu-Palmer similarity measure
        return  synset1.lin_similarity(synset2, ic)
    except:
        return -1

### Average Similarity using LIN

In [13]:
def lin_average(word1, word2):
    synsets_word1 = wn.synsets(word1, pos=wn.NOUN)
    synsets_word2 = wn.synsets(word2, pos=wn.NOUN)
    scores = []
    for s1 in synsets_word1:
        for s2 in synsets_word2:
            score = s1.lin_similarity(s2, ic)
            if score is not None:
                scores.append(score)

    return sum(scores) / len(scores) if scores else -1

### Run

In [14]:
df["wup_common"] = None
df["wup_average"] = None
df["lin_common"] = None
df["lin_average"] = None
for idx, row in df.iterrows():
    word1 = row["word1"]
    word2 = row["word2"]

    df.loc[idx, "wup_common"] = wup_common(word1, word2)
    df.loc[idx, "wup_average"] = wup_average(word1, word2)
    df.loc[idx, "lin_common"] = lin_common(word1, word2)
    df.loc[idx, "lin_average"] = lin_average(word1, word2)

    print('{:.2%}'.format(idx/464))
    display.clear_output(wait=True)

99.78%


In [15]:
df = df[df["wup_common"] != -1]

In [16]:
df.shape

(456, 11)

In [17]:
df.to_csv("data/456_complete_data.csv")